In [0]:
%load_ext autoreload
%autoreload 2

import matplotlib
from matplotlib import pyplot
pyplot.ion()
pyplot.style.use('dark_background')

import torch as th
from rl.algs.dqn import *

In [2]:
# GridWorld (debugging)
from rl.algs import experiment
from rl.algs import model
from rl.algs.envs import grid_world

BATCH_SIZE = 32
TRAIN_START_STEP = 3 * BATCH_SIZE
NUM_TRAIN_STEPS = 10000 # 17
REPLAY_SIZE = 100000
TARGET_NETWORK_UPDATE_FREQ = 500


env = environment.GridWorldWrapped(grid_shape=(4, 4, 1))
conv_specs = {
  model.ConvSpec(4, 1, 32),
}
fc_specs = (64,)
model_params = {
  'conv_specs': conv_specs,
  'fc_specs': fc_specs,
}

EPS_DECAY = util.Schedule((0, TRAIN_START_STEP, NUM_TRAIN_STEPS / 1, NUM_TRAIN_STEPS),
                          (1, 1, 0.0, 0.0))
EPS_RANDOM = util.Schedule((0,), (1.0,))
adam_params = {
  'fn': th.optim.Adam,
  'kwargs': {
    'lr': 0.005,
  }
}
rmsprop_params = {
  'fn': th.optim.RMSprop,
  'kwargs': {
    'lr': 0.00025,
    'momentum': 0.95,
    'alpha': 0.95,
    'eps': 0.01,
  }
}

policy_params = {
    'opt_params': rmsprop_params,
    'eps_sched': EPS_DECAY,  # TODO: change
    'target_update_freq': TARGET_NETWORK_UPDATE_FREQ,
}
exp = experiment.Experiment(env, model.QNetwork, model_params,
                            Dqn, policy_params)

print(exp.policy.model)

rb = ReplayBuffer(REPLAY_SIZE)
for step in range(TRAIN_START_STEP):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)

eps_rew = 0
for step in range(TRAIN_START_STEP, NUM_TRAIN_STEPS):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)
    
  experience_batch = rb.sample(BATCH_SIZE)
  m = exp.policy.update(experience_batch)
  
  eps_rew += sars.r
  if sars.s1 is None:
    m['r_per_eps'] = eps_rew
    eps_rew = 0
  if step % 100 == 0 or sars.s1 is None:
    for k, v in m.items():
      exp.plt.add_data(k, step, v)
      
# Expect about r_per_eps=-50 in pure random agent.

(1, 1, 32)
QNetwork(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [3]:
opt = grid_world.GridWorld((4, 4)).optimal_expected_reward()
print(f'optimal expected r: {opt}')

optimal expected r: 1.6666666666666665


In [4]:
env.show_model_policy(exp.policy.model)

[[ 0.          0.00568523 -0.83699536 -1.60975659]
 [ 0.03978002 -0.91353285 -1.64148402 -2.5964613 ]
 [-0.92274421 -1.7851789  -2.56734896 -3.49047685]
 [-1.85162091 -2.53062534 -3.3989234  -4.34690762]]
[['⟲' '←' '←' '←']
 ['↑' '←' '↑' '←']
 ['↑' '↑' '←' '↑']
 ['↑' '↑' '←' '↑']]
[[-0.00461879  0.         -0.15471113 -1.02429247]
 [-0.97769916 -0.02503818 -1.15532601 -1.91102231]
 [-1.81411695 -1.02783775 -1.87663317 -2.89154625]
 [-2.95696902 -2.01750994 -2.96133184 -3.78826165]]
[['→' '⟲' '←' '←']
 ['→' '↑' '←' '←']
 ['↑' '↑' '↑' '←']
 ['↑' '↑' '↑' '↑']]
[[-0.85882998  0.01165247  0.         -1.62070096]
 [-1.77377057 -1.07817566 -0.10253471 -0.78406572]
 [-2.84964347 -1.67143464 -0.95392597 -1.74539423]
 [-3.90538239 -2.88660598 -1.85839987 -2.86262798]]
[['→' '→' '⟲' '↓']
 ['↑' '↑' '↑' '←']
 ['↑' '→' '↑' '↑']
 ['→' '→' '↑' '←']]
[[-2.21822214 -1.1604929  -0.055518    0.        ]
 [-3.08058167 -2.08924222 -1.02089894 -0.04050872]
 [-3.85869861 -3.0868032  -2.00383282 -1.04326844]
 

In [5]:
# Debug the action-value function for a given state.
s = [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 2, 0]]
s = np.expand_dims(np.expand_dims(s, 0), -1)
print(exp.policy.model(util.to_variable(s)))
print(exp.policy.target_model(util.to_variable(s)))

tensor([[-1.8816, -2.0122,  0.0460, -1.7670]], device='cuda:0')
tensor([[-1.8920, -2.0103,  0.0656, -1.7762]], device='cuda:0')


In [6]:
# Examine the replay buffer.
terminals = rb.filter_by(sars_filter(r=0))
print(len(terminals))

# action-value at 's' was wrong. Search the replay buffer for occurrences of
# the correct transition. It's likely missing, which led the NN to overfit
# on missing data.
trans_s = rb.filter_by(sars_filter(s=s))
trans_s_a = rb.filter_by(sars_filter(s=s,a=2))
print(f'Num with (s): {len(trans_s)}. Num with (s, a): {len(trans_s_a)}')


1668
Num with (s): 110. Num with (s, a): 70


In [0]:
env.visualize(exp.policy, steps=20)